In [17]:
import numpy as np
import pandas as pd

In [18]:
# Load gates
x = np.array([[0,1],[1,0]])
y = np.array([[0,-1j],[1j,0]])
z = np.array([[1,0],[0,-1]])

h = np.sqrt(0.5)*np.array([[1,1],[1,-1]])
s = np.array([[1,0],[0,1j]])
I = np.eye(2)

cx = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
cz = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])
swap = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])

In [19]:
# MatMul and Tensor function
def mult(*ops):
    mat = ops[0]
    for op in ops[1:]:
        mat = np.matmul(mat,op)
    return mat

def tensor(*ops):
    tnsr = ops[0]
    for op in ops[1:]:
        tnsr = np.kron(tnsr,op)
    return tnsr

In [20]:
xxxx = tensor(x,x,x,x)
xxyy = tensor(x,x,y,y)
xyxy = tensor(x,y,x,y)
yxxy = tensor(y,x,x,y)

yyyy = tensor(y,y,y,y)
yyxx = tensor(y,y,x,x)
yxyx = tensor(y,x,y,x)
xyyx = tensor(x,y,y,x)

In [21]:
states = [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],
          [0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1],
          [1,0,0,0],[1,0,0,1],[1,0,1,0],[1,0,1,1],
          [1,1,0,0],[1,1,0,1],[1,1,1,0],[1,1,1,1]]

cb = np.eye(16)

def test_prod(op):
    for i in range(16):
        new = op@cb[i]
        ind = np.where(new == 1)
        assert len(ind) == 1
        val = ''
        if states[i] != states[ind[0][0]]:
            val = 'X'
        print(states[i],'->',states[ind[0][0]],val)

In [6]:
i = tensor(I,h,h,h)
ii = tensor(I,swap,I)
iii = mult(tensor(I,swap,I),
           tensor(I,I,cx),
           tensor(I,swap,I))
iv = tensor(I,I,cx)
v = mult(tensor(I,I,swap),tensor(I,swap,I),tensor(swap,I,I),
         tensor(cx,I,I),
         tensor(swap,I,I),tensor(I,swap,I),tensor(I,I,swap))
vi = mult(tensor(I,swap,I),tensor(swap,I,I),
          tensor(cx,I,I),
          tensor(swap,I,I),tensor(I,swap,I))
vii = mult(tensor(swap,I,I),tensor(cx,I,I),tensor(swap,I,I))
viii = mult(tensor(I,I,swap),tensor(I,swap,I),tensor(swap,I,I),
            tensor(cz,I,I),
            tensor(swap,I,I),tensor(I,swap,I),tensor(I,I,swap))
ix = mult(tensor(I,swap,I),
           tensor(I,I,cz),
           tensor(I,swap,I))
x = tensor(I,I,cz)
xi = tensor(h,h,h,h)

U = mult(i,ii,iii,iv,v,vi,vii,viii,ix,x,xi)
U[np.abs(U) < 1e-16] = 0
#print(U)


In [8]:
for b in range(16):
    if np.sum(np.equal(U[:,b],np.matmul(yyyy,U[:,b]))):
        print(b,': yes')

0 : yes
1 : yes
2 : yes
3 : yes
4 : yes
5 : yes
6 : yes
7 : yes
8 : yes
9 : yes
10 : yes
11 : yes
12 : yes
13 : yes
14 : yes
15 : yes


In [72]:
a = tensor(h,h,h,I)
b = mult(tensor(swap,I,I),tensor(I,swap,I),
           tensor(I,I,cx),
           tensor(I,swap,I),tensor(swap,I,I))
c = mult(tensor(I,swap,I),
           tensor(I,I,cx),
           tensor(I,swap,I))
d = tensor(I,I,cx)
e = mult(tensor(swap,I,I),tensor(I,swap,I),
           tensor(I,I,cz),
           tensor(I,swap,I),tensor(swap,I,I))
f = mult(tensor(I,swap,I),
           tensor(I,I,cz),
           tensor(I,swap,I))
g = tensor(I,I,cz)
had = tensor(h,h,h,h)
sw1 = mult(tensor(swap,swap),tensor(I,swap,I),tensor(swap,swap))
sw2 = tensor(I,swap,I)
altU = mult(a,b,c,d,e,f,g,had) #,sw1,sw2)
altU[np.abs(altU) < 1e-16] = 0

### Make dataframe


In [73]:
def get_eigvals(op,U,rounder=4):
    n = int(np.log2(op.shape[0]))
    eigvals = []
    for i in range(2**n):
        A = U[:,i]
        B = np.matmul(op,U[:,i])
        A = np.round(A,rounder)
        B = np.round(B,rounder)
        p = 0
        m = 0
        for o1,o2 in zip(A,B):
            if np.equal(o1,0) and np.equal(o2,0):
                p += 1
                m += 1
            elif np.equal(o1,o2):
                p += 1
            elif o1 == -o2:
                m += 1
        if p == len(U[:,i]):
            #print('{}{}{}{}'.format(*states[i]),': +')
            eigvals.append('+1')
        elif m == len(U[:,i]):
            #print('{}{}{}{}'.format(*states[i]),': -')
            eigvals.append('-1')
        #else:
        #    print('This is no eigenvalue')
    return eigvals

In [74]:
index = ['{}{}{}{}'.format(*states[i]) for i in range(16)]
ops = [xxxx,xxyy,xyxy,yxxy,yyyy,yyxx,yxyx,xyyx]
strs = ['xxxx','xxyy','xyxy','yxxy','yyyy','yyxx','yxyx','xyyx']
data = {}
alt_data = {}
for o,s in zip(ops,strs):
    data[s] = get_eigvals(o,U)
    alt_data[s] = get_eigvals(o,altU)

In [75]:
two_body = pd.DataFrame(data,columns=strs,index=index)
alt_two_body = pd.DataFrame(alt_data,columns=strs,index=index)

In [78]:
#print(two_body.to_latex())
#print(alt_two_body.to_latex())
print(two_body)
print(alt_two_body)

     xxxx xxyy xyxy yxxy yyyy yyxx yxyx xyyx
0000   +1   -1   -1   +1   -1   +1   +1   -1
0001   +1   -1   -1   -1   +1   -1   -1   -1
0010   +1   -1   +1   +1   +1   -1   +1   +1
0011   +1   -1   +1   -1   -1   +1   -1   +1
0100   +1   +1   -1   +1   +1   +1   -1   +1
0101   +1   +1   -1   -1   -1   -1   +1   +1
0110   +1   +1   +1   +1   -1   -1   -1   -1
0111   +1   +1   +1   -1   +1   +1   +1   -1
1000   -1   -1   -1   +1   -1   -1   -1   +1
1001   -1   -1   -1   -1   +1   +1   +1   +1
1010   -1   -1   +1   +1   +1   +1   -1   -1
1011   -1   -1   +1   -1   -1   -1   +1   -1
1100   -1   +1   -1   +1   +1   -1   +1   -1
1101   -1   +1   -1   -1   -1   +1   -1   -1
1110   -1   +1   +1   +1   -1   +1   +1   +1
1111   -1   +1   +1   -1   +1   -1   -1   +1
     xxxx xxyy xyxy yxxy yyyy yyxx yxyx xyyx
0000   +1   +1   +1   +1   -1   -1   -1   -1
0001   -1   +1   +1   +1   -1   +1   +1   +1
0010   +1   -1   +1   +1   +1   -1   +1   +1
0011   -1   -1   +1   +1   +1   +1   -1   -1
0100   +1 

xxxx -> 1. qubit

xxyy -> 2. qubit

xyxy -> 3. qubit

yxxy -> 4. qubit

yyyy -> Reversed parity except 1. qubit

yyxx -> Parity except 2. qubit

yxyx -> Parity except 3. qubit

xyyx -> Reversed parity except 4. qubit


In [28]:
y = tensor(I,x,y)
zzz = tensor(z,z,z)
xix = tensor(x,I,x)
zxy = tensor(z,x,y)

In [29]:
a = tensor(h,cx)
b = tensor(swap,I)
c = tensor(s,cz)
d = tensor(h,h,h)
U3 = mult(a,b,c,d)

In [23]:
U3

array([[ 0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0.5+0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.5j],
       [ 0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,
         0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
         0.5+0.j ,  0.5+0.j ],
       [ 0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,
         0. +0.j ,  0. +0.j ],
       [ 0. +0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.5j,
         0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         0. -0.5j,  0. -0.5j]])

In [38]:
for i in range(2**3):
    A = U3[:,i]
    B = np.matmul(ixy,U3[:,i])
    A = np.round(A,4)
    B = np.round(B,4)
    p = 0
    m = 0
    for o1,o2 in zip(A,B):
        if np.equal(o1,0) and np.equal(o2,0):
            p += 1
            m += 1
        elif np.equal(o1,o2):
            p += 1
        elif o1 == -o2:
            m += 1
    #if np.sum(np.equal(A,B)) == len(U3[:,i]):
    if p == len(U3[:,i]):
        print(i,': +')
    #if np.sum(np.equal(A,-B)) == len(U3[:,i]):
    elif m == len(U3[:,i]):
        print(i,': -')
    #for m1,m2 in zip(A,B):
    #    boo = ' '
    #    if m1 != m2:
    #        boo = 'X'
    #    if m1 == -m2:
    #        boo = 'V'
    #    print(boo,round(m1,4),round(m2,4))

# XX ZZ YY

In [7]:
xx = tensor(x,x)
yy = tensor(y,y)
zz = tensor(z,z)

U2 = mult(cx,tensor(h,I))

In [24]:
def test_basis(op,U,rounder=4):
    n = int(np.log2(op.shape[0]))
    for i in range(2**n):
        A = U[:,i]
        B = np.matmul(op,U[:,i])
        A = np.round(A,rounder)
        B = np.round(B,rounder)
        p = 0
        m = 0
        for o1,o2 in zip(A,B):
            if np.equal(o1,0) and np.equal(o2,0):
                p += 1
                m += 1
            elif np.equal(o1,o2):
                p += 1
            elif o1 == -o2:
                m += 1
        if p == len(U[:,i]):
            #print(i,': +')
            print('{}{}{}{}'.format(*states[i]),': +')
        elif m == len(U[:,i]):
            #print(i,': -')
            print('{}{}{}{}'.format(*states[i]),': -')

In [26]:
test_basis(yxyx,U)

0000 : +
0001 : -
0010 : +
0011 : -
0100 : -
0101 : +
0110 : -
0111 : +
1000 : -
1001 : +
1010 : -
1011 : +
1100 : +
1101 : -
1110 : +
1111 : -


# 

In [203]:
def com_mes(res):
    expval = [0,0,0]
    for state,val in res.items():
        e = [1,1,1]
        state = state[::-1]
        if state[0] == '1':
            e[0] *= -1
        if state[1] == '1':
            e[1] *= -1
        if sum([int(state[0]),int(state[1])])%2 == 1:
            e[2] *= -1
        for i in range(3):
            expval[i] += e[i]*val
        #print(state,e,val)
    return expval
        
def sgl_mes(res):
    expval = 0
    for state,val in res.items():
        state = state[::-1]
        e = 1
        for i in state:
            if i == '1':
                e *= -1
        expval += e*val
    return expval

In [204]:
print(com_mes(gc1))
print(sgl_mes(x1),sgl_mes(z1),sgl_mes(y1))

[-4, 4, -24]
4 -4 -24


In [206]:
print(com_mes(gc2))
print(sgl_mes(x2),sgl_mes(z2),sgl_mes(y2))

[-4, -1000, 4]
-24 -1000 -24


In [207]:
print(com_mes(gc3))
print(sgl_mes(x3),sgl_mes(z3),sgl_mes(y3))

[-4, -1000, 4]
-24 -1000 -24


In [321]:
def get_sngl(res):
    a = res['00'] if res.get('00') != None else 0
    b = res['10'] if res.get('10') != None else 0
    c = res['01'] if res.get('01') != None else 0
    d = res['11'] if res.get('11') != None else 0
    return (a+d)-(b+c)

def get_gc_x(res):
    a = res['00'] if res.get('00') != None else 0
    b = res['10'] if res.get('10') != None else 0
    c = res['01'] if res.get('01') != None else 0
    d = res['11'] if res.get('11') != None else 0
    return a-b+c-d#(a-b)+(d-c)

def get_gc_y(res):
    a = res['00'] if res.get('00') != None else 0
    b = res['10'] if res.get('10') != None else 0
    c = res['01'] if res.get('01') != None else 0
    d = res['11'] if res.get('11') != None else 0
    return (a+d)-(b+c)

def get_gc_z(res):
    a = res['00'] if res.get('00') != None else 0
    b = res['10'] if res.get('10') != None else 0
    c = res['01'] if res.get('01') != None else 0
    d = res['11'] if res.get('11') != None else 0
    return (a+d)-(b+c)

In [322]:
x1 = {'10': 498, '00': 502}
y1 = {'00': 244, '11': 244, '01': 258, '10': 254}
z1 = {'01': 502, '00': 498}

gc1 = {'01': 258, '10': 254, '00': 244, '11': 244}

print(com_mes(gc1))
print(sgl_mes(x1),sgl_mes(z1),sgl_mes(y1))

print(get_gc_x(gc1),get_gc_z(gc1),get_gc_y(gc1))

[-4, 4, -24]
4 -4 -24
-24 -24 -24


In [307]:
x2 = {'10': 254, '01': 258, '11': 244, '00': 244}
y2 = {'10': 254, '01': 258, '11': 244, '00': 244}
z2 = {'01': 1000}

gc2 = {'10': 498, '11': 502}

print(com_mes(gc2))
print(sgl_mes(x2),sgl_mes(z2),sgl_mes(y2))

print(get_gc_x(gc2),get_gc_z(gc2),get_gc_y(gc))

[-4, -1000, 4]
-24 -1000 -24
-1000 4 -24


In [288]:
x3 = {'10': 254, '00': 244, '01': 258, '11': 244}
y3 = {'10': 254, '00': 244, '01': 258, '11': 244}
z3 = {'10': 1000}

gc3 = {'10': 498, '11': 502}

print(com_mes(gc3))
print(sgl_mes(x3),sgl_mes(z3),sgl_mes(y3))

print(get_gc_x(gc3),get_gc_z(gc3),get_gc_y(gc3))

[-4, -1000, 4]
-24 -1000 -24
-4 4 4


In [289]:
x4 =  {'10': 498, '11': 502}
y4 = {'01': 258, '10': 254, '11': 244, '00': 244}
z4 = {'01': 502, '11': 498}

gc4 = {'01': 258, '10': 254, '11': 244, '00': 244}

print(com_mes(gc4))
print(sgl_mes(x4),sgl_mes(z4),sgl_mes(y4))

print(get_gc_x(gc4),get_gc_z(gc4),get_gc_y(gc4))

[-4, 4, -24]
4 -4 -24
-4 -24 -24


In [290]:
x5 = {'01': 8, '00': 510, '11': 10, '10': 472}
y5 = {'01': 61, '00': 47, '11': 432, '10': 460}
z5 = {'01': 278, '00': 513, '11': 80, '10': 129}

gc5 = {'01': 291, '00': 307, '11': 199, '10': 203}

print(com_mes(gc5))
print(sgl_mes(x5),sgl_mes(z5),sgl_mes(y5))

print(get_gc_x(gc5),get_gc_z(gc5),get_gc_y(gc5))

[20, 196, 12]
40 186 -42
20 12 12


In [308]:
x6 = {'00': 513, '10': 348, '01':  84, '11':  55}
y6 = {'00':  97, '10': 724, '01':  18, '11': 161}
z6 = {'00': 513, '10': 129, '01': 278, '11':  80}

#gc6 = {'00': 455, '10': 97, '01': 143, '11': 305}
gc6 = {'00': 684, '10': 179, '01': 27, '11': 110}

print(com_mes(gc6)) 
print(sgl_mes(x6),sgl_mes(z6),sgl_mes(y6))

print(get_gc_x(gc6),get_gc_z(gc6),get_gc_y(gc6))

[726, 422, 588]
136 186 -484
422 588 588


In [325]:
print(+684 + 179 + 27 + 110)
print(+684 - 179 + 27 + 110)
print(+684 + 179 - 27 + 110)
print(+684 + 179 + 27 - 110)
print(+684 - 179 - 27 + 110)
print(+684 - 179 + 27 - 110)
print(+684 + 179 - 27 - 110)
print(+684 - 179 - 27 - 110)
print(-684 + 179 + 27 + 110)
print(-684 - 179 + 27 + 110)
print(-684 + 179 - 27 + 110)
print(-684 + 179 + 27 - 110)
print(-684 - 179 - 27 + 110)
print(-684 - 179 + 27 - 110)
print(-684 + 179 - 27 - 110)
print(-684 - 179 - 27 - 110)

1000
642
946
780
588
422
726
368
-368
-726
-422
-588
-780
-946
-642
-1000


In [326]:
x6 = {'10': 268, '01': 139, '11': 503, '00': 90}
y6 = {'10': 268, '01': 139, '11': 503, '00': 90}
z6 = {'10': 144, '01': 302, '11': 70, '00': 484}

gc6 = {'10': 230, '01': 279, '11': 201, '00': 290}

print(com_mes(gc6)) 
print(sgl_mes(x6),sgl_mes(z6),sgl_mes(y6))

print(get_gc_x(gc6),get_gc_z(gc6),get_gc_y(gc6))

[40, 138, -18]
186 108 186
-18 -18 -18


In [331]:
230 - 279 - 201 + 290

40

In [9]:
for state in states:
    zero = np.array([1,0])
    one = np.array([0,1])
    if state[0] == 0:
        a = np.array([1,0])
    if state[0] == 1:
        a = np.array([0,1])
    for i in state[1:]:
        if i == 0:
            b = np.array([1,0])
        if i == 1:
            b = np.array([0,1])
        a = np.kron(a,b)
    print(state,a)

[0, 0, 0, 0] [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 0, 0, 1] [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 0, 1, 0] [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 0, 1, 1] [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 1, 0, 0] [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0, 1, 0, 1] [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0, 1, 1, 0] [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0, 1, 1, 1] [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[1, 0, 0, 0] [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[1, 0, 0, 1] [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[1, 0, 1, 0] [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[1, 0, 1, 1] [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[1, 1, 0, 0] [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[1, 1, 0, 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[1, 1, 1, 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[1, 1, 1, 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
